In [89]:
import tangram as tg
import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
import torch
from scipy import stats
from sklearn.metrics import r2_score

from time import time

t = !pwd
t = t[0]
%cd /mnt/win1/
from SC2Spa import PP
%cd $t

In [2]:
pct_offset = 0

def MinMaxNorm(Y):
    return (Y-Y.min(axis = 0))/(Y.max(axis = 0)-Y.min(axis = 0))/(1+pct_offset)

def ReMMNorm(Y, Y_pred):
    return (Y_pred*(Y.max(axis=0)-Y.min(axis=0)*(1+pct_offset))+Y.min(axis = 0))

def PolarTrans(Y):
    
    R = np.sqrt(np.square(Y[:,0]) + np.square(Y[:,1]))
    Theta = np.arctan(Y[:,1]/Y[:,0])
    RTheta = np.concatenate([R.reshape(-1,1), Theta.reshape(-1,1)], axis = 1)
    
    return RTheta

def RePolarTrans(RTheta):
    x = RTheta[:,0] * np.cos(RTheta[:,1])
    y = RTheta[:,0] * np.sin(RTheta[:,1])
    Y = np.concatenate([x.reshape(-1,1), y.reshape(-1,1)], axis = 1)
    return Y

In [84]:
adata = ad.read_h5ad('../Dataset/AdataMH1.h5ad')
adata.var_names_make_unique()

Y = adata.obs[['xcoord', 'ycoord']].values
YNorm = MinMaxNorm(Y)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [53]:
train_indices = []
test_indices = []
for i in range(5):
    train_index = pd.read_csv('CV_groups/index_train_' + str(i+1) + '.csv',
                              header = None, index_col = 0).values.flatten()
    test_index = pd.read_csv('CV_groups/index_test_' + str(i+1) + '.csv',
                             header = None, index_col = 0).values.flatten()
    
    train_indices.append(train_index)
    test_indices.append(test_index)

In [61]:
train_indices[0]

array([ 4680, 27793, 26831, ...,  7017, 32234, 33364])

In [60]:
adata[train_indices[0]]

View of AnnData object with n_obs × n_vars = 28279 × 20527
    obs: 'xcoord', 'ycoord', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ERCC', 'pct_counts_ERCC', 'leiden'
    var: 'n_cells', 'mt', 'ERCC', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap', 'log1p', 'hvg'
    obsm: 'X_pca', 'X_umap'

# Finetune number of genes

In [71]:
ad_maps = []

for i in np.array(range(1,36))*100:
    print(i)
    adata = ad.read_h5ad('../Dataset/AdataMH1.h5ad')
    adata.var_names_make_unique()

    sc.pp.normalize_total(adata, target_sum = 1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes = i)


    ad_sp = adata[train_indices[0]]
    ad_sc = adata[test_indices[0]]
    tg.pp_adatas(ad_sc, ad_sp, genes=adata.var_names[adata.var.highly_variable].tolist())

    ad_map = tg.map_cells_to_space(ad_sc, ad_sp, device = 'cuda')
    ad_maps.append(ad_map)
    torch.cuda.empty_cache()

100


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
INFO:root:97 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:19435 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 97 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.211, KL reg: 0.137
Score: 0.918, KL reg: 0.012
Score: 0.936, KL reg: 0.008
Score: 0.939, KL reg: 0.007
Score: 0.941, KL reg: 0.007
Score: 0.942, KL reg: 0.007
Score: 0.942, KL reg: 0.007
Score: 0.943, KL reg: 0.007
Score: 0.943, KL reg: 0.007
Score: 0.944, KL reg: 0.007


INFO:root:Saving results..


200


...

INFO:root:Saving results..


3500


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
INFO:root:3315 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:19435 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 3315 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.104, KL reg: 0.137
Score: 0.646, KL reg: 0.004
Score: 0.670, KL reg: 0.003
Score: 0.677, KL reg: 0.003
Score: 0.682, KL reg: 0.002
Score: 0.684, KL reg: 0.002
Score: 0.686, KL reg: 0.002
Score: 0.687, KL reg: 0.002
Score: 0.688, KL reg: 0.002
Score: 0.689, KL reg: 0.002


INFO:root:Saving results..


In [90]:
def MinMaxNorm(Y, Y_ref = Y):
    return (Y-Y_ref.min(axis = 0))/(Y_ref.max(axis = 0)-Y_ref.min(axis = 0))/(1+pct_offset)

def evaluate(pred, Y_true):

    pearsonr_x = stats.pearsonr(Y_true[:, 0], pred[:, 0])[0]
    pearsonr_y = stats.pearsonr(Y_true[:, 1], pred[:, 1])[0]
    rmse = np.sqrt(np.square(Y_true - pred).sum() / Y_true.shape[0])
    r2 = r2_score(Y_true, pred, multioutput='variance_weighted')
    
    return [pearsonr_x, pearsonr_y, rmse, r2]

for i in range(35):
    Y_pred = np.matmul(ad_maps[i].X, Y[train_indices[0]])
    Tangram = MinMaxNorm(Y_pred, Y_ref = Y)
    
    pd.DataFrame(Y_pred).to_csv('SI_Benchmarking/Tangram/pred_' + str(i) + '.csv', index = None)
    
    performance = evaluate(pred = Tangram,
                               Y_true = YNorm[test_indices[0]])
    print((i+1)*100, ':', performance)

100 : [0.16733936302543984, 0.2620920052597901, 0.3573855345969716, 0.0465985376586991]
200 : [0.26748712163230626, 0.3754840073928316, 0.3473617557793867, 0.09932962941604027]
300 : [0.3615003937625531, 0.4141813378348783, 0.34161163145836193, 0.12890168778370137]
400 : [0.3596541034399609, 0.46636381903314933, 0.33940710858696316, 0.1401083309865528]
500 : [0.3751353377011392, 0.48494762276879166, 0.3380946528797336, 0.14674571519586496]
600 : [0.3855157984084811, 0.5021588430702528, 0.33709083722869027, 0.15180488127864294]
700 : [0.38714169565257994, 0.47984515505530906, 0.33888588994169055, 0.1427473295308982]
800 : [0.39270617003840474, 0.5080913689252888, 0.3374777209179584, 0.14985679409731506]
900 : [0.3893368849492296, 0.5187320641085044, 0.33707725910508884, 0.1518732110279948]
1000 : [0.391214448558735, 0.5262319349209161, 0.3370673661299965, 0.1519229941198184]
1100 : [0.4003103463576151, 0.5320857124475046, 0.3364139861831669, 0.1552076755092115]
1200 : [0.410270341643108

In [91]:
ad_maps = []

for i in np.array(range(4,20))*900:
    print(i)
    adata = ad.read_h5ad('../Dataset/AdataMH1.h5ad')
    adata.var_names_make_unique()

    sc.pp.normalize_total(adata, target_sum = 1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes = i)


    ad_sp = adata[train_indices[0]]
    ad_sc = adata[test_indices[0]]
    tg.pp_adatas(ad_sc, ad_sp, genes=adata.var_names[adata.var.highly_variable].tolist())

    ad_map = tg.map_cells_to_space(ad_sc, ad_sp, device = 'cuda')
    ad_maps.append(ad_map)
    torch.cuda.empty_cache()

3600


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
INFO:root:3409 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:19435 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 3409 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.104, KL reg: 0.137
Score: 0.641, KL reg: 0.004
Score: 0.665, KL reg: 0.003
Score: 0.673, KL reg: 0.003
Score: 0.677, KL reg: 0.002
Score: 0.679, KL reg: 0.002
Score: 0.681, KL reg: 0.002
Score: 0.683, KL reg: 0.002
Score: 0.684, KL reg: 0.002
Score: 0.685, KL reg: 0.002


INFO:root:Saving results..


4500


...

INFO:root:Saving results..


17100


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
INFO:root:16597 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:19435 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 16597 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.123, KL reg: 0.137
Score: 0.373, KL reg: 0.001
Score: 0.399, KL reg: 0.001
Score: 0.409, KL reg: 0.000
Score: 0.414, KL reg: 0.000
Score: 0.418, KL reg: 0.000
Score: 0.420, KL reg: 0.000
Score: 0.422, KL reg: 0.000
Score: 0.423, KL reg: 0.000
Score: 0.424, KL reg: 0.000


INFO:root:Saving results..


In [92]:
def MinMaxNorm(Y, Y_ref = Y):
    return (Y-Y_ref.min(axis = 0))/(Y_ref.max(axis = 0)-Y_ref.min(axis = 0))/(1+pct_offset)

def evaluate(pred, Y_true):

    pearsonr_x = stats.pearsonr(Y_true[:, 0], pred[:, 0])[0]
    pearsonr_y = stats.pearsonr(Y_true[:, 1], pred[:, 1])[0]
    rmse = np.sqrt(np.square(Y_true - pred).sum() / Y_true.shape[0])
    r2 = r2_score(Y_true, pred, multioutput='variance_weighted')
    
    return [pearsonr_x, pearsonr_y, rmse, r2]

for i in range(16):
    Y_pred = np.matmul(ad_maps[i].X, Y[train_indices[0]])
    Tangram = MinMaxNorm(Y_pred, Y_ref = Y)
    
    pd.DataFrame(Y_pred).to_csv('SI_Benchmarking/Tangram/pred_' + str(i) + '.csv', index = None)
    
    performance = evaluate(pred = Tangram,
                               Y_true = YNorm[test_indices[0]])
    print((i+4)*900, ':', performance)

3600 : [0.36529961515135534, 0.5142540260970104, 0.34380052845043135, 0.11770269402015238]
4500 : [0.36949446118427137, 0.5161920112914472, 0.3451190905216201, 0.11092204905885021]
5400 : [0.35905736294512286, 0.5206937629867276, 0.3458232242184795, 0.10729044249605495]
6300 : [0.3782442663608973, 0.5431158061360156, 0.34517893507980607, 0.11061368541453011]
7200 : [0.39437665908124164, 0.5514137879537183, 0.3448798096292904, 0.11215446779291725]
8100 : [0.398174270561543, 0.5613800544872382, 0.34473310945500657, 0.11290962590497777]
9000 : [0.40344091102429513, 0.5625101086574485, 0.3448299021661608, 0.11241140896995393]
9900 : [0.410937477534061, 0.5763136246632539, 0.34423469836775034, 0.11547286133807513]
10800 : [0.4154574118141797, 0.5819982317603916, 0.34402338318655157, 0.11655849647631894]
11700 : [0.42461907584639685, 0.5895925722942909, 0.3434134111501876, 0.11968849816999]
12600 : [0.4152814171934159, 0.5932559469281619, 0.34336671769197025, 0.11992787142981877]
13500 : [0.

# Running code

In [97]:
ad_maps = []
sta = time()

adata = ad.read_h5ad('../Dataset/AdataMH1.h5ad')
adata.var_names_make_unique()

sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes = 8000)

for i in range(5):

    print(i)
    ad_sp = adata[train_indices[i]]
    ad_sc = adata[test_indices[i]]
    tg.pp_adatas(ad_sc, ad_sp, genes=None)

    ad_map = tg.map_cells_to_space(ad_sc, ad_sp, device = 'cuda')
    ad_maps.append(ad_map)
    torch.cuda.empty_cache()

end = time()
print((end - sta)/60.0, 'mins consumed')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


0


INFO:root:19435 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:19435 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 19435 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.120, KL reg: 0.137
Score: 0.375, KL reg: 0.001
Score: 0.400, KL reg: 0.000
Score: 0.409, KL reg: 0.000
Score: 0.413, KL reg: 0.000
Score: 0.417, KL reg: 0.000
Score: 0.419, KL reg: 0.000
Score: 0.421, KL reg: 0.000
Score: 0.422, KL reg: 0.000
Score: 0.423, KL reg: 0.000


INFO:root:Saving results..


...

4


INFO:root:19375 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:19375 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 19375 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.121, KL reg: 0.138
Score: 0.375, KL reg: 0.001
Score: 0.400, KL reg: 0.000
Score: 0.409, KL reg: 0.000
Score: 0.414, KL reg: 0.000
Score: 0.417, KL reg: 0.000
Score: 0.419, KL reg: 0.000
Score: 0.421, KL reg: 0.000
Score: 0.422, KL reg: 0.000
Score: 0.423, KL reg: 0.000


INFO:root:Saving results..


164.94758803049723 mins consumed


In [98]:
len(ad_sc.uns['training_genes'])

19375

In [99]:
i = 0
for i in range(5):
    Y_pred = np.matmul(ad_maps[i].X, Y[train_indices[i]])
    pd.DataFrame(Y_pred).to_csv('SI_Benchmarking/Tangram/pred_' + str(i) + '.csv', index = None)

In [64]:
Y_pred

array([[3107.59604494, 3229.44047571],
       [3558.652212  , 3088.45137994],
       [3199.65489801, 3185.65921776],
       ...,
       [2871.82632982, 3491.64458997],
       [3146.28042943, 3242.12208494],
       [3191.7016737 , 2892.28896944]])

In [36]:
pd.DataFrame(Y_pred).to_csv('SI_Benchmarking/Tangram/pred_' + str(4) + '.csv', index = None)

In [27]:
len(ad_sc.uns['training_genes'])

19435

In [15]:
help(tg.pp_adatas)

Help on function pp_adatas in module tangram.mapping_utils:

pp_adatas(adata_sc, adata_sp, genes=None)
    Pre-process AnnDatas so that they can be mapped. Specifically:
    - Remove genes that all entries are zero
    - Find the intersection between adata_sc, adata_sp and given marker gene list, save the intersected markers in two adatas
    - Calculate density priors and save it with adata_sp
    
    Args:
        adata_sc (AnnData): single cell data
        adata_sp (AnnData): spatial expression data
        genes (List): Optional. List of genes to use. If `None`, all genes are used.
    
    Returns:
        update adata_sc by creating `uns` `training_genes` `overlap_genes` fields 
        update adata_sp by creating `uns` `training_genes` `overlap_genes` fields and creating `obs` `rna_count_based_density` & `uniform_density` field



In [ ]:
adata